In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!apt install -y gdal-bin
!pip install geopandas rasterio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-gdal python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 5,055 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.6.4+dfsg-1~jammy0 [1,027 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.6.4+dfsg-1~jammy0 [561 kB]
Fetched 5,055 kB in 2s (2,571 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 126315 files and direc

In [ ]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling
from rasterio.io import MemoryFile

In [ ]:
base_dir = '/content/drive/My Drive/THESIS/GHS_POP'
aoi_path = '/content/drive/My Drive/THESIS/mum_extent.geojson'
dst_crs = 'EPSG:32643'

aoi = gpd.read_file(aoi_path).to_crs(dst_crs)

pop_files = ['POP_E2015.tif', 'POP_E2020.tif', 'POP_E2025.tif']

In [ ]:

for fname in pop_files:
    input_path = os.path.join(base_dir, fname)
    year = fname.split('_')[1].replace('.tif', '')
    reproj_path = os.path.join(base_dir, f'GHS_POP_{year}_UTM43N.tif')
    clipped_path = os.path.join(base_dir, f'GHS_POP_{year}_UTM43N_Clipped.tif')

    if not os.path.exists(input_path):
        print(f"File not found: {fname}")
        continue

    print(f"Processing: {fname}")

    # Reproject
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(reproj_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

    print(f"Reprojected to UTM Zone 43N: {reproj_path}")

    # Clip and resample
    with rasterio.open(reproj_path) as src:
        out_image, out_transform = mask(src, aoi.geometry, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        with MemoryFile() as memfile:
            with memfile.open(**out_meta) as temp_dst:
                temp_dst.write(out_image)

            with memfile.open() as resample_src:
                transform, width, height = calculate_default_transform(
                    resample_src.crs, resample_src.crs,
                    resample_src.width, resample_src.height,
                    *resample_src.bounds,
                    resolution=(30, 30))

                resampled_meta = resample_src.meta.copy()
                resampled_meta.update({
                    'transform': transform,
                    'width': width,
                    'height': height
                })

                with rasterio.open(clipped_path, 'w', **resampled_meta) as dst:
                    for i in range(1, resample_src.count + 1):
                        reproject(
                            source=rasterio.band(resample_src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=resample_src.transform,
                            src_crs=resample_src.crs,
                            dst_transform=transform,
                            dst_crs=resample_src.crs,
                            resampling=Resampling.bilinear
                        )

    print(f"Clipped and resampled to 30m: {clipped_path}")


Processing: POP_E2015.tif
Reprojected to UTM Zone 43N: /content/drive/My Drive/THESIS/GHS_POP/GHS_POP_E2015_UTM43N.tif
Clipped and resampled to 30m: /content/drive/My Drive/THESIS/GHS_POP/GHS_POP_E2015_UTM43N_Clipped.tif
Processing: POP_E2020.tif
Reprojected to UTM Zone 43N: /content/drive/My Drive/THESIS/GHS_POP/GHS_POP_E2020_UTM43N.tif
Clipped and resampled to 30m: /content/drive/My Drive/THESIS/GHS_POP/GHS_POP_E2020_UTM43N_Clipped.tif
Processing: POP_E2025.tif
Reprojected to UTM Zone 43N: /content/drive/My Drive/THESIS/GHS_POP/GHS_POP_E2025_UTM43N.tif
Clipped and resampled to 30m: /content/drive/My Drive/THESIS/GHS_POP/GHS_POP_E2025_UTM43N_Clipped.tif
